## Imports   

In [1]:
import sys
print(sys.version_info)

import os
import cv2
import time
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt
import configparser
import tensorflow as tf
import time
import itertools
from glob import glob
from pathlib import Path
from ast import literal_eval
from tensorflow.keras.utils import to_categorical, plot_model
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.layers import Input, Flatten, Dense, Lambda, Cropping2D, Dropout, Conv2D, concatenate, TimeDistributed, CuDNNLSTM, LSTM, BatchNormalization
from tensorflow.keras.utils import Sequence
from tensorflow.keras.losses import mean_squared_error, binary_crossentropy

import tensorflow.keras.backend as K

# import pydotplus

tf.logging.set_verbosity(tf.logging.INFO)
print(tf.__version__)

sys.version_info(major=3, minor=5, micro=6, releaselevel='final', serial=0)
1.10.0


## Helper functions


In [2]:
hlc_one_hot = { 1: [1,0,0,0,0,0], 2:[0,1,0,0,0,0], 3:[0,0,1,0,0,0], 4:[0,0,0,1,0,0], 5:[0,0,0,0,1,0], 6:[0,0,0,0,0,1]}
#hlc_one_hot = { 1: [1,0,0,0], 2:[0,1,0,0], 3:[0,0,1,0], 4:[0,0,0,1]}

def flatten_list(items_list):
    """ Removes the episode dimension from a list of data points """
    ret = []
    for items in items_list:
        for item in items:
            ret.append(item)        
    return ret      

# Custom loss function 
def weighted_mse(y_true, y_pred, weight_mask):
    """ 
    Custom loss fucntion, different weigted loss to steer/throttle/brake 
    Used when all outputs is evaluated by same loss function 
    """
    return K.mean(K.square(y_pred - y_true)*weight_mask, axis=-1)

def root_mean_squared_error(y_true, y_pred):
    """ Custom loss function, RMSE """
    return K.sqrt(K.mean(K.square(y_pred - y_true)))
    
def steer_loss():
    """ Loss function for steering, RMSE """
    def custom(y_true, y_pred):
        return root_mean_squared_error(y_true, y_pred)
    return custom

def get_hlc_one_hot(hlc):
    """ One-hot encode HLC values """
    return hlc_one_hot[hlc]

def sine_encode(angle):
    """ Encode steering angle as sine wave """
    angle_max = 1
    N = 10
    ret = []
    
    for i in range(1,N+1,1):
        Y = np.sin(((2*np.pi*(i-1))/(N-1))-((angle*np.pi)/(2*angle_max)))
        ret.append(Y)
    
    return np.array(ret)

def create_input_dict():
    return {
        "forward_imgs": [],
        "info_signals": [],
        "hlcs" : [],
        "environments": []
    }

def create_target_dict():
    return {
        "steer" : [],
        "throttle": [],
        "brake" : []
    }

def split_dict(dictionary, split_pos):
    """ Split data into training and validation """
    train_dict = {}
    val_dict = {}
    
    for key in dictionary:
        train_dict[key] = dictionary[key][:split_pos]
        val_dict[key] = dictionary[key][split_pos:]
    
    return train_dict, val_dict

def adjust_hlcs(hlcs, info_signals):
    """ 
    Randomly adjust HLC backwards, 
    such that the HLC data is given before the car is in the intersection 
    """
    
    # Iterate over all episodes 
    for e in range(len(hlcs)): 
        active_change_hlc = None
        changed = 0
        adjust_num = random.randint(20, 40)
        # Iterate over all hlcs in episode 
        for i in range(len(hlcs[e])-2,-1, -1):
            next_hlc = np.argmax(hlcs[e][i+1]) + 1 
            current_hlc = np.argmax(hlcs[e][i]) +1 
            if active_change_hlc == False:
                active_change_hlc = None
                continue

            if active_change_hlc is None:
                if next_hlc == 1 or next_hlc == 2 or next_hlc == 3:                     
                    if current_hlc == 4: 
                        active_change_hlc = hlcs[e][i+1]
            if active_change_hlc is not None: 
                hlcs[e][i] = active_change_hlc
                if info_signals[e][i][2] != 0: 
                    changed += 1
                if changed == adjust_num:
                    active_change_hlc = False
                    changed = 0 
    return hlcs 

#### BALANCING DATA - Helper functions #### 
def get_hlc_dist_label(hlc): 
    " Get distributions of HLC in data "
    if hlc == 1: 
        return "left"
    elif hlc == 2:
        return "right"
    elif hlc == 3:
        return "straight"
    elif hlc == 4:
        return "follow_lane"
    elif hlc == 5:
        return "change_lane_left"
    elif hlc == 6:
        return "change_lane_right"
    
def get_speed_dist_label(speed): 
    """ Get distributions of speed in data """

    if speed < 0.001: 
        return "low_speed"
    else: 
        return "high_speed"
    
def get_speed_limit_dist_label(speed_limit): 
    """ Get distributions of speed limits in data """

    speed_limit = round(speed_limit, 1)
    if speed_limit == 0.3: 
        return "30km/h"
    elif speed_limit == 0.6: 
        return "60km/h"
    elif speed_limit == 0.9: 
        return "90km/h"
    
def get_percentage_dist(dist): 
    """ Get distribution of data in percentage """
    N = dist["traffic_light"]["red"] + dist["traffic_light"]["green"]
    for dist_key in dist:
        for key, dist_val in dist[dist_key].items(): 
            dist[dist_key][key] = dist_val/N 
    return dist 

def get_drop_num(tot_num, num, keep_fraction):
    """ Calculates how many datasamples one should drop to get the right amount of data samples """
    return int((num - keep_fraction*tot_num)/(1-keep_fraction))


def shuffle_data(inputs_flat, targets_flat):
    # Shuffle
    indices = np.arange(len(inputs_flat["forward_imgs"]))
    np.random.shuffle(indices)
    
    for key in inputs_flat: 
        inputs_flat[key] = np.array(inputs_flat[key])[indices]
        
    for key in targets_flat: 
        targets_flat[key] = np.array(targets_flat[key])[indices]

    return inputs_flat, targets_flat
         

## Data Augmentations 
- Lightness
- Hue 
- Gaussian Blur
- Shadows 
- Rain

In [3]:
def show_image(lab_img): 
    bgr_img = cv2.cvtColor(lab_img, cv2.COLOR_LAB2BGR) 
    rgb_img = cv2.cvtColor(bgr_img, cv2.COLOR_BGR2RGB) 
    plt.imshow(rgb_img)
    plt.show()

    
def generate_shadow_coordinates(imshape, no_of_shadows=1):
    vertices_list=[]
    for index in range(no_of_shadows):
        vertex=[]
        for dimensions in range(np.random.randint(3,15)): ## Dimensionality of the shadow polygon
            vertex.append(( imshape[1]*np.random.uniform(),imshape[0]//3+imshape[0]*np.random.uniform()))
        vertices = np.array([vertex], dtype=np.int32) ## single shadow vertices 
        vertices_list.append(vertices)
    return vertices_list ## List of shadow vertices

def generate_random_lines(imshape,slant,drop_length):    
    drops=[]    
    for i in range(1500): 
        ## If You want heavy rain, try increasing this        
        if slant<0:            
            x= np.random.randint(slant,imshape[1])        
        else:            
            x= np.random.randint(0,imshape[1]-slant)        
        y= np.random.randint(0,imshape[0]-drop_length)        
        
        drops.append((x,y))    
    return drops 

def change_light(bgr_img, light_coeff, debug=False):
    
    lab_img = cv2.cvtColor(bgr_img, cv2.COLOR_BGR2LAB) 
    lab_img[:,:,0] = np.minimum(lab_img[:,:,0]*light_coeff, 255)
    if debug: 
        show_image(lab_img)
    
    return np.array(lab_img, dtype = np.uint8)
    
def change_hue(bgr_img, hue_coeff, debug=False): 
    hsv_img = cv2.cvtColor(bgr_img, cv2.COLOR_BGR2HSV) 
    hsv_img[:,:,0] = (hsv_img[:,:,0]*hue_coeff)%255   
    
    bgr_img = cv2.cvtColor(hsv_img, cv2.COLOR_HSV2BGR) 
    lab_img = cv2.cvtColor(bgr_img, cv2.COLOR_BGR2LAB)
    
    if debug: 
        show_image(lab_img)
    return np.array(lab_img, dtype = np.uint8)
           

def add_rain(bgr_img, debug=False):     
    bgr_img = np.copy(bgr_img)
    imshape = bgr_img.shape    
    slant_extreme = 10   
    slant= 0 #np.random.randint(-slant_extreme,slant_extreme)     
    drop_length=7
    drop_width=1   
    drop_color=(200,200,200) ## a shade of gray   
    rain_drops= generate_random_lines(imshape,slant,drop_length)        
    for rain_drop in rain_drops:        
        cv2.line(bgr_img,(rain_drop[0],rain_drop[1]),(rain_drop[0]+slant,rain_drop[1]+drop_length),drop_color,drop_width)    
    bgr_img= cv2.blur(bgr_img,(3,3)) ## rainy view are blurry         
    
    lab_img = cv2.cvtColor(bgr_img, cv2.COLOR_BGR2LAB)

    if debug: 
        show_image(lab_img)
    
    return np.array(lab_img, dtype = np.uint8)

    

def add_shadow(bgr_img,no_of_shadows=1, debug=False):
    lab_img = cv2.cvtColor(bgr_img, cv2.COLOR_BGR2LAB) 
    mask = np.zeros_like(lab_img) 
    img_shape = lab_img.shape
    vertices_list= generate_shadow_coordinates(img_shape, no_of_shadows) #3 getting list of shadow vertices
    for vertices in vertices_list: 
        cv2.fillPoly(mask, vertices, 255) ## adding all shadow polygons on empty mask, single 255 denotes only red channel
    
    lab_img[:,:,0][mask[:,:,0]==255] = lab_img[:,:,0][mask[:,:,0]==255]*0.2   ## if red channel is hot, image's "Lightness" channel's brightness is lowered 
    
    if debug: 
        show_image(lab_img)
    
    return np.array(lab_img, dtype = np.uint8)

def gaussian_blur(bgr_img, amount=3, debug=False):
    bgr_img = cv2.GaussianBlur(bgr_img,(amount,amount),0)
    lab_img = cv2.cvtColor(bgr_img, cv2.COLOR_BGR2LAB)
    if debug: 
        show_image(lab_img)
    
    return np.array(lab_img, dtype = np.uint8)

print_aug = False  

if print_aug: 
    test_path = "dataset/etron/Town04/ClientAP/cars_no_rain_noise5/2019-06-18_18-23-14/imgs/forward_center_rgb_00000082.png"
    #test_path = "dataset/etron/Town04/ClientAP/cars_no_rain_noise5/2019-06-18_18-17-14/imgs/forward_right_rgb_00000062.png"
    bgr_img = cv2.imread(test_path)
    print("Original image")
    show_image(cv2.cvtColor(bgr_img, cv2.COLOR_BGR2LAB))

    print("Gaussian")
    blur_img = gaussian_blur(bgr_img, amount=11, debug=True)

    print("Rain")
    rain_img = add_rain(bgr_img, debug=True)

    print("Shadows")
    shadow_img = add_shadow(bgr_img, debug=True)

    print("Lightness")
    light_img = change_light(bgr_img, light_range[0], debug=True)

    print("Hue")
    hue_img = change_hue(bgr_img, hue_range[1], debug=True)


## Load driving logs
- Loads data from all csv file in a lits of folders 
- Stores the data in a input dictionary and a target dictionary 
- Normalizes and coverts data to correct format 
- Does not use side cameraes for lane change data


In [4]:
def load_driving_logs(dataset_folders, steering_correction):
    """ 
    input: 
        dataset_folders: list of paths to folders to load data from 
        steering_correction: float, adjusts steering angles of forward facing side cameras 
        
    Loads data from all csv file in a lits of folders 
    Stores the data in a input dictionary and a target dictionary 
    Normalizes and coverts data to correct format 
    Does not use side cameraes for lane change data
    
    return: 
        inputs: dictionary of all input data (image paths, info signals, HLCs, control signals)
        targets: dictionary of all target data (steer, throttle, brake)
    """
    img_paths_center = []
    
    
    inputs = create_input_dict()
    
    targets = create_target_dict()
    # Loads data 
    for folder in dataset_folders:
        print("Checking folder: {:s}...".format(folder))
        folder_path = Path("dataset") / folder
        for episode in glob(str(folder_path / "*")):
            
            
            temp_forward = {"center": [], "left": [], "right": []}
            temp_info_signals =  {"center": [], "left": [], "right": []}
            temp_hlcs =  {"center": [], "left": [], "right": []}
            temp_environments =  {"center": [], "left": [], "right": []}

            temp_steer =  {"center": [], "left": [], "right": []}
            temp_throttle = {"center": [], "left": [], "right": []}
            temp_brake = {"center": [], "left": [], "right": []}
                
            episode_path = Path(episode)

            df = pd.read_csv(str(episode_path / "driving_log.csv"))
            for index, row in df.iterrows():
                if index == 0: 
                    continue 
                    
                speed = 0#float(row["Velocity"]) * 3.6 / 100
                speed_limit = 0#float(row["SpeedLimit"]) * 3.6 / 100
                traffic_light = 0#int(row["TrafficLight"])
                environment = 0#int(row["Environment"])
                
                #controls = literal_eval(row["Controls"])
                    
                steer = row["angle"]
                throttle = row["speed"]
                brake = 0#controls[2]
                
                """hlc = get_hlc_one_hot(row["HLC"])"""
                
                hlc = row["high_level_command"]
                if hlc == 0 or hlc == -1:
                    hlc = 4
                hlc = get_hlc_one_hot(hlc)
                
                temp_forward["center"].append(str(episode_path / row["image_path"]))              
                temp_steer["center"].append(steer)
                temp_throttle["center"].append(throttle)
                temp_brake["center"].append(brake)
                temp_info_signals["center"].append([speed, speed_limit, traffic_light])
                temp_hlcs["center"].append(hlc)
                temp_environments["center"].append(environment)
                
                # Only use the left/right shifted images for non lange-change data
                #if row["HLC"] != 5 and row["HLC"] != 6:
                #temp_forward["left"].append(str(episode_path / row["ForwardLeft"]))
                #temp_forward["right"].append(str(episode_path / row["ForwardRight"])) 
                
                temp_forward["left"].append(str(episode_path / row["image_path"]))
                temp_forward["right"].append(str(episode_path / row["image_path"])) 
                
                temp_throttle["left"].append(throttle)
                temp_throttle["right"].append(throttle)

                temp_steer["left"].append(steer+steering_correction)
                temp_steer["right"].append(steer-steering_correction)

                temp_brake["left"].append(brake)
                temp_brake["right"].append(brake)

                temp_info_signals["left"].append([speed, speed_limit, traffic_light])
                temp_info_signals["right"].append([speed, speed_limit, traffic_light])

                temp_hlcs["left"].append(hlc)
                temp_hlcs["right"].append(hlc)
                
                temp_environments["left"].append(environment)
                temp_environments["right"].append(environment)

                    
            inputs["forward_imgs"].append(temp_forward["center"])
            #inputs["forward_imgs"].append(temp_forward["left"])
            #inputs["forward_imgs"].append(temp_forward["right"])
            
            inputs["info_signals"].append(temp_info_signals["center"])
            #inputs["info_signals"].append(temp_info_signals["left"])
            #inputs["info_signals"].append(temp_info_signals["right"])
            
            inputs["hlcs"].append(temp_hlcs["center"])
            #inputs["hlcs"].append(temp_hlcs["left"])
            #inputs["hlcs"].append(temp_hlcs["right"])
            
            inputs["environments"].append(temp_environments["center"])
            #inputs["environments"].append(temp_environments["left"])
            #inputs["environments"].append(temp_environments["right"])
            
            targets["steer"].append(temp_steer["center"])
            #targets["steer"].append(temp_steer["left"])
            #targets["steer"].append(temp_steer["right"])
            
            targets["throttle"].append(temp_throttle["center"])
            #targets["throttle"].append(temp_throttle["left"])
            #targets["throttle"].append(temp_throttle["right"])
             
            targets["brake"].append(temp_brake["center"])
            #targets["brake"].append(temp_brake["left"])
            #targets["brake"].append(temp_brake["right"])
            
            


    print("Done, {:d} episode(s) loaded.".format(len(inputs["forward_imgs"])))

    return(inputs, targets)


## Plot Data 

In [5]:
def plot_data(dist, title=""):
    """ Plots distribution of HLC, speed and traffic lights """
    print("Plotting...")
    tot_num = 0
    for key in dist["environment"]:
        tot_num += dist["environment"][key]
        
    fig = plt.figure(figsize=(16,6))

    # HLC
    labels =["Follow Lane", "Left", "Right", "Straight", "Change Lane Left", "Change Lane Right"]
    sizes = [dist["hlc"]["follow_lane"], dist["hlc"]["left"], dist["hlc"]["right"], dist["hlc"]["straight"], dist["hlc"]["change_lane_left"], dist["hlc"]["change_lane_right"], ]
    
    ax1 = fig.add_subplot(1,4,1)
    wedges, texts, autotexts = ax1.pie(sizes, autopct='%.1f%%')
    for autotext in autotexts:
        autotext.set_color('white')
        autotext.set_fontweight('bold')

    ax1.set_title("Distrubution of HLC")
    ax1.legend(wedges,labels, loc="best")
    ax1.axis('equal')
    

    # Speed
    labels =["Low Speed", "High Speed"]
    sizes = [dist["speed"]["low_speed"], dist["speed"]["high_speed"]]
    
    ax2 = fig.add_subplot(1,4,2)
    wedges, texts, autotexts = ax2.pie(sizes, autopct='%.1f%%')
    for autotext in autotexts:
        autotext.set_color('white')
        autotext.set_fontweight('bold')
    ax2.set_title("Distrubution of speed")
    ax2.legend(wedges,labels, loc="best")
    ax2.axis('equal')

    # Traffic Light
    labels =["Red", "Green"]
    sizes = [dist["traffic_light"]["red"], dist["traffic_light"]["green"]]
    
    ax3 = fig.add_subplot(1,4,3)
    wedges, texts, autotexts = ax3.pie(sizes, autopct='%.1f%%')
    for autotext in autotexts:
        autotext.set_color('white')
        autotext.set_fontweight('bold')
    ax3.set_title("Distrubution of traffic light")
    ax3.legend(wedges,labels, loc="best")
    ax3.axis('equal')
    
    # Environment
    labels =["Town 1", "Town 4"]
    sizes = [dist["environment"]["town1"], dist["environment"]["town4"]]
    
    ax4 = fig.add_subplot(1,4,4)
    wedges, texts, autotexts = ax4.pie(sizes, autopct='%.1f%%')
    for autotext in autotexts:
        autotext.set_color('white')
        autotext.set_fontweight('bold')

    ax4.set_title("Distrubution of environments")
    ax4.legend(wedges,labels, loc="best")
    ax4.axis('equal')
    
    fig.suptitle(title + ": " + str(tot_num) + " sequences", fontsize=18)
    plt.show()

## Balance Data 

- Get distribution of HLC, speed, speed limit, and traffic lights 
- Balance data for LSTM 

In [6]:
def get_dist(inputs): 
    """ 
        input:
            inputs: dictionary of input data 
                
        return: 
            dist: dictionary of distributions for HLC, speed, speed limit, and traffic lights 
    """
    dist = {
        "hlc": {
            "left": 0,
            "right": 0, 
            "straight": 0,
            "follow_lane": 0,
            "change_lane_left": 0, 
            "change_lane_right": 0
        },
        "environment": {
            "town1": 0,
            "town4": 0
        },
        "speed": {
            "low_speed": 0, 
            "high_speed": 0,
        }, 
        "speed_limit": {
            "30km/h": 0,
            "60km/h": 0,
            "90km/h": 0
        }, 
        "traffic_light": {
            "red": 0, 
            "green": 0
        }
    }
    
    # HLC distribution
    for hlcs in inputs["hlcs"]: 
        
        follow_lane = True 
        # Iterate over all HLC in sequence 
        for hlc in hlcs: 
            hlc_value = np.argmax(hlc) + 1 
            # Only mark sequence as follow lane if all hlcs are follow lane  
            if hlc_value != 4:
                follow_lane = False 
                hlc_label = get_hlc_dist_label(hlc_value)
                dist["hlc"][hlc_label] += 1 
                break
        if follow_lane: 
            dist["hlc"]["follow_lane"] += 1  
            
    # Environment distribution 
    for environment in inputs["environments"]: 
        if environment[0] == 0: 
            dist["environment"]["town4"] +=1 
        if environment[0] == 1: 
            dist["environment"]["town1"] +=1
            
    # Info signals distribution 
    for info_signals in inputs["info_signals"]: 
        red_light = True 
        speeds = []
        speed_limits = []
        
        # For each sequence 
        for info_signal in info_signals: 
            speeds.append(info_signal[0])                
            speed_limits.append(info_signal[1]*10)
            
            traffic_light = info_signal[2]
            if traffic_light == 1: 
                red_light = False 
                
                
        # Update dist after each sequence 
        
        # Max speed 
        speed_label = get_speed_dist_label(np.max(np.array(speeds)))
        dist["speed"][speed_label] += 1 

        # Most frequent speed limit 
        most_frequent_speed_limit = np.argmax(np.bincount(speed_limits))/10
        speed_limit_label = get_speed_limit_dist_label(most_frequent_speed_limit) 
        dist["speed_limit"][speed_limit_label] += 1 
        
        # Mark sequemce as green light if it has any frame with green light 
        if red_light: 
            dist["traffic_light"]["red"] += 1
        else: 
            dist["traffic_light"]["green"] += 1
        
        
    return dist   
    


def balance_speed(inputs, targets, dist, target_key, target_fraction):
    
    """ Balance speed such that target fraction is correct """
    
    # Calculate number of data samples that should be droped 
    tot_num = dist["speed"]["low_speed"] + dist["speed"]["high_speed"]
    target_speed_num =  dist["speed"][target_key] 
    drop_target_speed_num = get_drop_num(tot_num, target_speed_num, target_fraction)

    inputs_bal = create_input_dict()
    targets_bal = create_target_dict()
    
    # Balance low speed 
    dropped = 0
    for i in range(len(inputs["info_signals"])): 
        info_signals = inputs["info_signals"][i]
        speeds = []
        for info_signal in info_signals:
            speeds.append(info_signal[0])
        speed = np.max(speeds)
            
        # Drop        
        if get_speed_dist_label(speed) == target_key and (drop_target_speed_num>dropped):
            dropped += 1 
            continue
        
        # Keep 
        for key in inputs_bal: 
            inputs_bal[key].append(inputs[key][i])
        for key in targets_bal: 
            targets_bal[key].append(targets[key][i])
        
    return inputs_bal, targets_bal 
    
def balance_traffic_light(inputs, targets, dist, target_key, target_fraction):
    """ Balance traffic lights such that target fraction is correct """
    
    # Calculate number of data samples that should be droped 
    tot_num = dist["traffic_light"]["red"] + dist["traffic_light"]["green"]
    target_speed_num =  dist["traffic_light"][target_key] 
    drop_target_traffic_light_num = get_drop_num(tot_num, target_speed_num, target_fraction)

    inputs_bal = create_input_dict()
    targets_bal = create_target_dict() 

    # Balance traffic lights 
    dropped = 0
    for i in range(len(inputs["info_signals"])): 
        info_signals = inputs["info_signals"][i]
        
        # Mark episode as green light if there is at least one green traffic light in sequence 
        traffic_lights = []
        for info_signal in info_signals:
            traffic_light = "red" if info_signal[2] == 0 else "green"
            traffic_lights.append(traffic_light)
        traffic_light = "green" if "green" in traffic_lights else "red" 
            
        # Drop        
        if traffic_light == target_key and (drop_target_traffic_light_num>dropped):
            dropped += 1
            continue
        
        # Keep 
        for key in inputs_bal: 
            inputs_bal[key].append(inputs[key][i])
        for key in targets_bal: 
            targets_bal[key].append(targets[key][i])     

    return inputs_bal, targets_bal 

def balance_hlc(inputs, targets, dist, lane_change):
    """ Balance HLC (except follow lane) such that target fraction is correct """

    # Get correct dist data TODO: combine to one dict ? 
    if lane_change: 
        num_hlcs = {
            "change_lane_left": dist["hlc"]["change_lane_left"], 
            "change_lane_right": dist["hlc"]["change_lane_right"]
        }
    else: 
        num_hlcs = {
            "straight": dist["hlc"]["straight"],
            "left": dist["hlc"]["left"],
            "right": dist["hlc"]["right"]
        }
    freqs = []
    for key in num_hlcs : 
        freqs.append(num_hlcs[key])
    num = np.min(freqs)
    
    inputs_bal = create_input_dict()
    targets_bal = create_target_dict() 
    
    for i in range(len(inputs["hlcs"])): 
        # Find current hlc 
        hlcs = inputs["hlcs"][i]
        current_hlc = "follow_lane"
        for hlc in hlcs: 
            hlc_type = get_hlc_dist_label(np.argmax(hlc)+1)
            if hlc_type != "follow_lane": 
                current_hlc = hlc_type
                break
        # Drop        
        if current_hlc != "follow_lane" and (num_hlcs[current_hlc]>num):
            num_hlcs[current_hlc]-=1
            continue
        
        # Keep 
        for key in inputs_bal: 
            inputs_bal[key].append(inputs[key][i])
        for key in targets_bal: 
            targets_bal[key].append(targets[key][i])

    return inputs_bal, targets_bal 
        

def balance_follow_lane(inputs, targets, dist, target_fraction):
    """ Balance follow lane such that target fraction is correct """
    
    tot_num = 0 
    target_hlc_num = 0
    for hlc_key in dist["hlc"]:
        tot_num += dist["hlc"][hlc_key] 
            
        
    target_hlc_num =  dist["hlc"]["follow_lane"] 
    drop_target_hlc_num = get_drop_num(tot_num, target_hlc_num, target_fraction)

    inputs_bal = create_input_dict()
    targets_bal = create_target_dict() 
    
    dropped = 0
    
    # Find current hlc 
    for i in range(len(inputs["hlcs"])): 
        hlcs = inputs["hlcs"][i]
        current_hlc = "follow_lane"
        for hlc in hlcs: 
            hlc_type = get_hlc_dist_label(np.argmax(hlc)+1)
            if hlc_type != "follow_lane": 
                current_hlc = hlc_type
                break
            
        # Drop        
        if current_hlc == "follow_lane" and (drop_target_hlc_num>dropped):
            dropped +=1
            continue
        
        # Keep 
        for key in inputs_bal: 
            inputs_bal[key].append(inputs[key][i])
        for key in targets_bal: 
            targets_bal[key].append(targets[key][i])

    return inputs_bal, targets_bal 
    

def balance_data_lstm(inputs, targets, follow_lane_frac=0.7, red_light_frac=0.01, low_speed_frac=0.05,debug=False, drop=True):
    """ Balance dataset for LSTM data """
    print("Balancing data:")
    
    # Get distribution 
    dist = get_dist(inputs)
    
    inputs_bal = inputs
    targets_bal = targets
    dist_bal = dist 
    
    # Balance traffic lights 
    print("   - Balancing traffic lights")
    inputs_bal, targets_bal = balance_traffic_light(inputs_bal, targets_bal, dist_bal,"red", red_light_frac)
    dist_bal = get_dist(inputs_bal)
    
    # Balance low speed 
    print("   - Balancing low speed")
    inputs_bal, targets_bal = balance_speed(inputs_bal, targets_bal, dist_bal,"low_speed", low_speed_frac)
    """
    dist_bal = get_dist(inputs_bal)
    
    # Ballance hlc - left, left, straight 
    print("   - Balancing HLCs")
    inputs_bal, targets_bal = balance_hlc(inputs_bal, targets_bal, dist_bal,False)    
    dist_bal = get_dist(inputs_bal)
    
    # Ballance hlc - follow lane 
    print("   - Balancing LANEFOLLOW")
    inputs_bal, targets_bal = balance_follow_lane(inputs_bal, targets_bal, dist_bal, follow_lane_frac)
    """
    
    # Shuffle
    inputs_bal, targets_bal = shuffle_data(inputs_bal, targets_bal)
    
    if drop: 
        inputs_bal, targets_bal = drop_follow_lane(inputs_bal,targets_bal, 3)
    dist_bal = get_dist(inputs_bal)
        
    return inputs_bal, targets_bal, dist_bal

## Drop random data 
Randomly drop data that is follow lane without brake values or turns 

In [7]:
def drop_follow_lane(inputs, targets, keep_every): 
    print("Dropping...")
    
    inputs_dropped = create_input_dict()
    targets_dropped = create_target_dict() 
    

    for i in range(len(inputs["forward_imgs"])): 
        
        
        follow_lane = True 
        # Iterate over all HLC in sequence 
        for hlc in inputs["hlcs"][i]: 
            hlc_value = np.argmax(hlc) + 1 
            # Only mark sequence as follow lane if all hlcs are follow lane  
            if hlc_value != 4:
                follow_lane = False 
                break
                
        brake = targets["brake"][i]
        steer = targets["steer"][i]
        
        # Only drop if FOLLOW_LANE, no brake, and no turn
        if follow_lane and brake == 0 and abs(steer)<0.1:  
            # Drop based on drop_rate 
            
            if np.random.randint(keep_every) != 0: 
                continue

        # Keep 
        for key in inputs_dropped: 
            inputs_dropped[key].append(inputs[key][i])
        for key in targets_dropped: 
            targets_dropped[key].append(targets[key][i])

    return inputs_dropped, targets_dropped 
    

# LSTM

## Prepare dataset format

In [ ]:
def get_episode_sequences(data, sampling_interval, seq_length):
    sequences = []
    slices = []
    for o in range(sampling_interval+1):
        slices.append(data[o::sampling_interval+1])
    for s in slices:
        for o in range(0,len(s)):
            if o + seq_length <= len(s):
                sequences.append(s[o:o+seq_length])
    return sequences

def prepare_dataset_lstm(inputs, targets, sampling_interval, seq_length):
    
    inputs_flat = create_input_dict()
    targets_flat = create_target_dict()
    
    for e in range(len(inputs["forward_imgs"])):
        [inputs_flat["forward_imgs"].append(sequence) for sequence in get_episode_sequences(inputs["forward_imgs"][e],sampling_interval, seq_length)]
        [inputs_flat["info_signals"].append(sequence) for sequence in get_episode_sequences(inputs["info_signals"][e],sampling_interval, seq_length)]
        [inputs_flat["hlcs"].append(sequence) for sequence in get_episode_sequences(inputs["hlcs"][e],sampling_interval, seq_length)]
        [inputs_flat["environments"].append(sequence) for sequence in get_episode_sequences(inputs["environments"][e],sampling_interval, seq_length)]
        [targets_flat["steer"].append(sequence[-1]) for sequence in get_episode_sequences(targets["steer"][e],sampling_interval, seq_length)]
        [targets_flat["throttle"].append(sequence[-1]) for sequence in get_episode_sequences(targets["throttle"][e],sampling_interval, seq_length)]
        [targets_flat["brake"].append(sequence[-1]) for sequence in get_episode_sequences(targets["brake"][e],sampling_interval, seq_length)]

    
    # Shuffle
    inputs_flat, targets_flat = shuffle_data(inputs_flat, targets_flat)
    
    return(inputs_flat, targets_flat)
        

## Define model

In [ ]:
def get_lstm_model(seq_length, print_summary=True):
    forward_image_input = Input(shape=(seq_length, 162, 217, 3), name="forward_image_input")
    info_input = Input(shape=(seq_length,3), name="info_input")
    hlc_input = Input(shape=(seq_length,6), name="hlc_input")

    x = TimeDistributed(Cropping2D(cropping=((50,0),(0,0))))(forward_image_input)
    x = TimeDistributed(Lambda(lambda x: ((x/255.0) - 0.5)))(x)
    x = TimeDistributed(Conv2D(24,(5,5),strides=(2,2), activation="relu"))(x)
    x = TimeDistributed(Conv2D(36,(5,5),strides=(2,2), activation="relu"))(x)
    x = TimeDistributed(Conv2D(48,(5,5),strides=(2,2), activation="relu"))(x)
    x = TimeDistributed(Conv2D(64,(3,3),strides=(2,2), activation="relu"))(x)
    x = TimeDistributed(Conv2D(64,(3,3), activation="relu"))(x)
    x = TimeDistributed(Conv2D(64,(3,3), activation="relu"))(x)
    conv_output = TimeDistributed(Flatten())(x)

    x = concatenate([conv_output, info_input, hlc_input])
    
    x = TimeDistributed(Dense(100, activation="relu"))(x)
    x = CuDNNLSTM(10, return_sequences = False)(x)
    steer_pred = Dense(10, activation="tanh", name="steer_pred")(x)
    
    x = TimeDistributed(Cropping2D(cropping=((50,0),(0,0))))(forward_image_input)
    x = TimeDistributed(Lambda(lambda x: ((x/255.0) - 0.5)))(x)
    x = TimeDistributed(Conv2D(24,(5,5),strides=(2,2), activation="relu"))(x)
    x = TimeDistributed(Conv2D(36,(5,5),strides=(2,2), activation="relu"))(x)
    x = TimeDistributed(Conv2D(48,(5,5),strides=(2,2), activation="relu"))(x)
    x = TimeDistributed(Conv2D(64,(3,3),strides=(2,2), activation="relu"))(x)
    x = TimeDistributed(Conv2D(64,(3,3), activation="relu"))(x)
    x = TimeDistributed(Conv2D(64,(3,3), activation="relu"))(x)
    conv_output = TimeDistributed(Flatten())(x)

    x = concatenate([conv_output, info_input, hlc_input])
    
    x = TimeDistributed(Dense(100, activation="relu"))(x)
    x = CuDNNLSTM(10, return_sequences = False)(x)
    throtte_pred = Dense(1, name="throttle_pred")(x)
    brake_pred = Dense(1, name="brake_pred")(x)
    
    model = Model(inputs=[forward_image_input, info_input, hlc_input], outputs=[steer_pred,throtte_pred,brake_pred])

    if print_summary: 
        model.summary()
    
    return model

# Training

## Define generator

In [ ]:
def get_path(image_path):
    return "./dataset/2019-10-09T11:07:52/episode1/images/" + image_path.split("/")[-1]

print(get_path("/home/kia/data_dump/2019-10-09T11:07:52/images/55972.jpg"))
class generator(Sequence):
    def __init__(self, inputs, targets, batch_size, validation=False):
        self.inputs = inputs
        self.targets = targets
        self.batch_size = batch_size
        self.validation = validation 
        
        random.seed() 
        # Convert to np array
        for key in inputs: 
            inputs[key] = np.array(self.inputs[key])

        for key in targets: 
            targets[key] = np.array(self.targets[key])

    def __len__(self):
        return int(np.ceil(len(self.inputs["forward_imgs"]) / float(self.batch_size)))

    def __getitem__(self, idx):
        subset = np.arange(idx * self.batch_size,min((idx + 1) * self.batch_size,len(self.inputs["forward_imgs"])))
        forward_imgs = []
        steer_pred = np.array([sine_encode(steer) for steer in self.targets["steer"][subset]])
        read_images = [[cv2.resize(cv2.imgread(get_path(path)), fx=0.5, fy=0.5) for path in seq] for seq in self.inputs["forward_imgs"][subset]]
        if not self.validation: 
            augment = 2 #random.randint(0,3)
            if augment == 0: 
                augment_type = random.randint(0,4)
                
                # Lightness 
                if augment_type == 0: 
                    light_coeff = random.random()*0.9+0.5
                    for seq in read_images:
                        forward_imgs.append([change_light(image), light_coeff=light_coeff) for image in seq])
                # Hue 
                elif augment_type == 1:
                    hue_coeff = random.random()*1.4 + 0.2
                    for seq in read_images:
                        forward_imgs.append([change_hue(image), hue_coeff=hue_coeff) for image in seq])
                # Rain 
                elif augment_type == 2:
                    for seq in read_images:
                        forward_imgs.append([add_rain(image) for image in seq])
                
                # Shadows 
                elif augment_type == 3:
                    for seq in read_images:
                        forward_imgs.append([add_shadow(image) for image in seq])
                # Gaussian blur 
                elif augment_type == 4:
                    for seq in read_images:
                        forward_imgs.append([gaussian_blur(image) for image in seq])
            else: 
                for seq in read_images:
                    forward_imgs.append([cv2.cvtColor(image,cv2.COLOR_BGR2LAB) for image in seq])
        else:
            for seq in read_images:
                forward_imgs.append([cv2.cvtColor(image,cv2.COLOR_BGR2LAB) for image in seq])

        return {
            "forward_image_input": np.array(forward_imgs), 
            "info_input": self.inputs["info_signals"][subset],
            "hlc_input": self.inputs["hlcs"][subset]
        }, {
            "steer_pred": steer_pred,
            "throttle_pred": self.targets["throttle"][subset],
            "brake_pred": self.targets["brake"][subset]
        }
            


## Parameters

In [14]:
#model_name = "test4_new_autopilot"
val_split = 0.8
adjust_hlc = False 
balance_data = False 


epochs_list = [15]
dataset_folders_lists = [["2019-10-09T11:07:52"]]
"""dataset_folders_lists = [([
    "etron/Town01/ClientAP/no_cars_no_rain",
    "etron/Town01/ClientAP/no_cars_no_rain_noise15",
    "etron/Town01/ClientAP/no_cars_rain_noise10",
    "etron/Town01/ClientAP/cars_no_rain_noise15",
    "etron/Town01/ClientAP/cars_rain_noise10",    
], [
    "etron/Town04/ClientAP/no_cars_no_rain", 
    "etron/Town04/ClientAP/no_cars_rain_noise5", 
    "etron/Town04/ClientAP/cars_no_rain_noise5", 
    "etron/Town04/ClientAP/cars_rain_noise5", 
    "etron/Town01/ClientAP/brake_all_weather_noise15",
    "etron/Town01/ClientAP/brake_all_weather",
])]"""


steering_corrections = [0.05]

batch_sizes = [32]

sampling_interval = [2]

seq_length = [10]

parameter_permutations = itertools.product(epochs_list, 
                                           dataset_folders_lists, 
                                           steering_corrections, 
                                           batch_sizes,
                                           sampling_interval,
                                           seq_length)  
        

## Training loop

In [ ]:
# Train a new model for each parameter permutation, and save the best models
model_name = input("Name of model test: ").strip()
balance_data = True if input("Balance data y/[n]: ").lower() == "y" else False 
#drop_data = True if input("Drop data y/[n]: ").lower() == "y" else False 
print("")

for parameters in parameter_permutations:
    
    # Get parameters
    epochs, dataset_folders, steering_correction, batch_size, sampling_interval, seq_length = parameters
    parameters_string = ("epochs:\t\t\t{}\ndataset folders:\t{}\nsteering correction:\t{}\nbatch size:\t\t{}\nsampling interval:\t{}\nseq lenght: \t\t{}\n\n"
                         .format(epochs, str(dataset_folders), steering_correction, batch_size, sampling_interval, seq_length))
    
    
    #town1_dataset_folders, town4_dataset_folders = dataset_folders

    # Prepare for logging
    timestamp = time.strftime("%Y-%m-%d_%H-%M-%S", time.localtime(time.time()))
    path = Path('models') / model_name / timestamp
    if not os.path.exists(str(path)):
        os.makedirs(str(path))
        
    # Save parmeters to disk
    with open(str(path / "parameters.txt"), "w") as text_file:
        text_file.write(parameters_string)
    
    # Save config file to disk 
    model_name = "lstm"
    config = configparser.ConfigParser()
    config["ModelConfig"] = {'Model': model_name,'sequence_length': seq_length,'sampling_interval': sampling_interval}
    with open(str(path/'config.ini'), 'w') as configfile:
        config.write(configfile)
    
    # Load drive logs and paths
    #inputs1, targets1 = load_driving_logs(town1_dataset_folders, steering_correction)
    #inputs4, targets4 = load_driving_logs(town4_dataset_folders, steering_correction)
    inputs, targets = load_driving_logs(dataset_folders, steering_correction)

    # Adjust hlcs on town 1 data 
    if adjust_hlc: 
        inputs["hlcs"] = inputs(inputs1["hlcs"], inputs["info_signals"])
    
    # Prepare dataset to model format 
    #inputs_flat1, targets_flat1 = prepare_dataset_lstm(inputs1, targets1, sampling_interval, seq_length)
    #inputs_flat4, targets_flat4 = prepare_dataset_lstm(inputs4, targets4, sampling_interval, seq_length)
    inputs_flat, targets_flat = prepare_dataset_lstm(inputs, targets, sampling_interval, seq_length)

    num_before1 = len(inputs_flat["forward_imgs"])
    # Balance data 
    #if balance_data:
    #    # Plot data before balancing 
    #    title_before = "Town 1 - Before Balancing"
    #    plot_data(get_dist(inputs_flat1), title=title_before)

        # Balance data 
    #    inputs_flat1, targets_flat1, dist_bal1 = balance_data_lstm(inputs_flat1, targets_flat1)

        # Plot data after balancing 
    #    title_after = "Town 1 - After Balancing"
    #    plot_data(dist_bal1, title=title_after)


    inputs_flat_dict = create_input_dict() 
    targets_flat_dict = create_target_dict() 

    for key in inputs_flat: 
        #inputs_flat[key] = np.concatenate((inputs_flat1[key],inputs_flat4[key]))
        inputs_flat_dict[key] = inputs_flat[key] 

    for key in targets_flat: 
        targets_flat_dict[key] = targets_flat[key]
                
    # Shuffle data 
    inputs_flat, targets_flat = shuffle_data(inputs_flat, targets_flat)
    
    # Plot final data set 
    #plot_data(get_dist(inputs_flat), title="Final distribution")
           
    
    # Split into val and train
    split_pos = int(val_split*len(inputs_flat["forward_imgs"]))
    inputs_train, inputs_val = split_dict(inputs_flat, split_pos)
    targets_train, targets_val = split_dict(targets_flat, split_pos)

    # Print training info 
    train_num = len(inputs_train["forward_imgs"])
    val_num = len(inputs_val["forward_imgs"])
    print("Initiate training loop with the following parameters:")
    print("---")
    print(parameters_string)
    print("---")
    print("Training set size: " + str(train_num))
    print("Validation set size: " + str(val_num))
 
    # Get model
    model = get_lstm_model(seq_length ,print_summary=False)

    
    # Compile model 
    model.compile(loss=[steer_loss(), mean_squared_error, mean_squared_error] , optimizer=Adam())

    checkpoint_val = ModelCheckpoint(str(path / ('{epoch:02d}_s{val_steer_pred_loss:.4f}_t{val_throttle_pred_loss:.4f}_b{val_brake_pred_loss:.4f}.h5')), monitor='val_loss', verbose=1, save_best_only=False,mode="min")
    
    # Create image of model architecture 
    plot_model(model, str(path/'model.png'))
    
    steps = int(train_num/batch_size)
    steps_val = int(val_num/batch_size)
     
    # Train model
    history_object = model.fit_generator(
        generator(inputs_train, targets_train, batch_size),
        validation_data=generator(inputs_val, targets_val, batch_size, validation=True),
        epochs=epochs,
        verbose=1,
        callbacks=[checkpoint_val],
        steps_per_epoch=steps,
        validation_steps = steps_val,
        use_multiprocessing = True,
        workers=10
    )
    
    # Prepare plot and save it to disk
    fig = plt.figure()
    ax = fig.add_subplot(111)
    
    print("History", history_object.history)
    ax.plot(history_object.history['steer_pred_loss'], color="blue")
    ax.plot(history_object.history['val_steer_pred_loss'], color="blue", linestyle="--")
    ax.plot(history_object.history['throttle_pred_loss'], color="green")
    ax.plot(history_object.history['val_throttle_pred_loss'], color="green", linestyle="--")
    ax.plot(history_object.history['brake_pred_loss'], color="red")
    ax.plot(history_object.history['val_brake_pred_loss'], color="red", linestyle="--")

    ax.set_title("Mean squared loss of: throttle, brake and steer")
    ax.set_xlabel("epochs")
    ax.set_ylabel("mse")

    lgd = ax.legend(['steer loss', 
               'steer validation loss',
               'throttle loss', 
               'throttle validation loss', 
               'brake loss', 
               'brake validation loss'], bbox_to_anchor=(1.1, 1.05))
    
    plt.show()
    fig.savefig(str(path / 'loss.png'), bbox_extra_artists=(lgd,), bbox_inches='tight')
    print('\n\n\n\n')

Name of model test: initial_spurv_model
Balance data y/[n]: n

Checking folder: 2019-10-09T11:07:52...
Done, 1 episode(s) loaded.
Initiate training loop with the following parameters:
---
epochs:			15
dataset folders:	['2019-10-09T11:07:52']
steering correction:	0.05
batch size:		32
sampling interval:	2
seq lenght: 		10


---
Training set size: 65455
Validation set size: 16364
Epoch 1/15
1464/2045 [====================>.........] - ETA: 27:16 - loss: 0.1886 - steer_pred_loss: 0.1868 - throttle_pred_loss: 0.0017 - brake_pred_loss: 9.2069e-05

array([[[ 28, 129, 137],
        [ 16, 130, 133],
        [ 15, 130, 133],
        ...,
        [111, 136,  91],
        [103, 137,  91],
        [ 95, 138,  90]],

       [[ 39, 129, 138],
        [ 14, 130, 134],
        [ 23, 130, 136],
        ...,
        [118, 136,  91],
        [110, 136,  91],
        [102, 137,  91]],

       [[ 36, 129, 141],
        [ 26, 130, 139],
        [ 27, 128, 139],
        ...,
        [126, 134,  93],
        [117, 136,  91],
        [109, 136,  91]],

       ...,

       [[ 47, 129, 132],
        [ 46, 130, 130],
        [ 42, 130, 130],
        ...,
        [ 34, 127, 130],
        [ 30, 128, 128],
        [ 28, 129, 124]],

       [[ 46, 130, 130],
        [ 52, 129, 130],
        [ 45, 130, 130],
        ...,
        [ 36, 127, 130],
        [ 39, 128, 128],
        [ 39, 129, 125]],

       [[ 46, 130, 130],
        [ 51, 129, 131],
        [ 45, 130, 130],
        ...,
        [ 33, 125, 130],
        [ 39, 128, 128],
        [ 36, 129, 124]]